In [ ]:
# We won't need TensorFlow here
!pip uninstall -y tensorflow
!pip install sentencepiece
# Install `transformers` from master
!pip install git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers'
# transformers version at notebook update --- 2.11.0
# tokenizers version at notebook update --- 0.8.0rc1

In [ ]:
import sentencepiece as spm 
spm.SentencePieceTrainer.Train('--input=/content/drive/MyDrive/100mbout/out2006_3.txt --vocab_size=52000 --model_prefix=sp10m.cased.v3 --character_coverage=0.99995 --model_type=unigram')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import XLNetConfig

config = XLNetConfig(
    vocab_size=52000,
    d_model = 32,
    #max_position_embeddings=514,
    n_head=4,
    n_layer=6,
    #d_inner=32,
    #d_model=32,
    dropout=0.1,
    dropatt=0.1,
    attn_type="bi", 
    clamp_len=-1,
    same_length=False,
    mem_len = 1024
    #type_vocab_size=1,
)

In [ ]:
from transformers import XLNetModel
model = XLNetModel(config=config)

In [ ]:
from transformers import XLNetTokenizerFast

tokenizer = XLNetTokenizerFast("/content/xlnet/sp10m.cased.v3.model", max_len=512)

In [ ]:
!pip install torch

In [ ]:
import io
import os
import math
import torch
import warnings
from tqdm.notebook import tqdm
from transformers import (
                          CONFIG_MAPPING,
                          MODEL_FOR_MASKED_LM_MAPPING,
                          MODEL_FOR_CAUSAL_LM_MAPPING,
                          PreTrainedTokenizer,
                          TrainingArguments,
                          AutoConfig,
                          AutoTokenizer,
                          AutoModelWithLMHead,
                          AutoModelForCausalLM,
                          AutoModelForMaskedLM,
                          LineByLineTextDataset,
                          TextDataset,
                          DataCollatorForLanguageModeling,
                          DataCollatorForWholeWordMask,
                          DataCollatorForPermutationLanguageModeling,
                          PretrainedConfig,
                          Trainer,
                          set_seed,
                          )
model = AutoModelWithLMHead.from_config(config)

In [ ]:
model.num_parameters()
# => 84 million parameters

3346112

In [ ]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="/content/drive/MyDrive/100mbout/out2006_3.txt",
    block_size=128,
)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:124: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


CPU times: user 4min 26s, sys: 8.36 s, total: 4min 34s
Wall time: 1min 40s


In [ ]:
from transformers import DataCollatorForLanguageModeling, DataCollatorForPermutationLanguageModeling

#data_collator = DataCollatorForLanguageModeling(
#    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
#)
plm_probability=1 / 6
max_span_length= 5
data_collator = DataCollatorForPermutationLanguageModeling(
                                          tokenizer=tokenizer,
                                          plm_probability=plm_probability,
                                          max_span_length=max_span_length,
                                          )

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./xlnet",
    max_steps = 1000,
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

### Start training

In [ ]:
%%time
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


Step,Training Loss
500,10.428700
1000,9.973000


CPU times: user 5min 46s, sys: 2min 24s, total: 8min 11s
Wall time: 4min 7s


TrainOutput(global_step=1000, training_loss=10.20087744140625, metrics={'train_runtime': 246.1444, 'train_samples_per_second': 4.063, 'total_flos': 164450108325888.0, 'epoch': 0.06, 'init_mem_cpu_alloc_delta': 0, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 28536832, 'train_mem_gpu_alloc_delta': 48717312, 'train_mem_cpu_peaked_delta': 0, 'train_mem_gpu_peaked_delta': 11243222016})

In [ ]:
trainer.save_model("./xlnet")